In [2]:
import pandas as pd

import requests 

import os

import json

import googleapiclient.discovery
import googleapiclient.errors

In [3]:
df_youtubers = pd.read_csv('data/get_youtubers_output/youtubers.csv')

In [4]:
chiave_yt = json.load(open('key.json', 'r'))['youtube']['key']
youtube = googleapiclient.discovery.build('youtube', 'v3', developerKey=chiave_yt)

In [20]:
quotas = 0

In [29]:
def get_videos_id(items):
    ids = []
    for item in items:
        #print(item['contentDetails'])
        try:
            ids.append(item['contentDetails']['upload']['videoId'])
        except KeyError:
            #print('Not upload')
            continue
    return ids

In [62]:
youtuber_videos = []
for idx, youtuber in enumerate(df_youtubers.values):
    print("___________________________________")
    print("{} Youtuber: {}".format(idx, youtuber[0]))
    channel_id = youtuber[1]
    videos = []
    next_page_token = " "
    while True:
        if quotas < 9000:
            request = youtube.activities().list(
                part="contentDetails",
                channelId=channel_id,
                maxResults=50,
                pageToken = next_page_token
            )
            response = request.execute()
            quotas += 3
            ids = get_videos_id(response['items'])
            print('Got {} new videos'.format(len(ids)))
            for vid in ids:
                if vid not in videos:
                    videos.append(vid)
            try:
                next_page_token = response['nextPageToken']
            except KeyError:
                break
            if len(videos) > 24:
                break
        else:
            print("Quota giornaliera superata")
    #print("Channel: {}, videos: {}".format(channel_id, videos))
    for vid in videos:
        youtuber_videos.append([channel_id,vid])

___________________________________
0 Youtuber: jeffreestar
Got 24 new videos
Got 27 new videos
___________________________________
1 Youtuber: Jaclyn Hill
Got 50 new videos
___________________________________
2 Youtuber: James Charles
Got 50 new videos
___________________________________
3 Youtuber: Manny Mua
Got 18 new videos
Got 18 new videos
___________________________________
4 Youtuber: Laura Lee
Got 50 new videos
___________________________________
5 Youtuber: Tati
Got 17 new videos
Got 21 new videos
___________________________________
6 Youtuber: KathleenLights
Got 50 new videos
___________________________________
7 Youtuber: Christen Dominique
Got 24 new videos
Got 25 new videos
___________________________________
8 Youtuber: Jackie Aina
Got 50 new videos
___________________________________
9 Youtuber: Casey Holmes
Got 46 new videos
___________________________________
10 Youtuber: Nicole Guerriero
Got 50 new videos
___________________________________
11 Youtuber: Desi Perkins


In [63]:
df_youtuber_videos = pd.DataFrame(youtuber_videos,columns=['channel_id','video_id'])

In [64]:
df_youtuber_videos

,channel_id,video_id
0,UCkvK_5omS-42Ovgah8KRKtg,WyUyJKm-MiY
1,UCkvK_5omS-42Ovgah8KRKtg,5mUQo6lQG-U
2,UCkvK_5omS-42Ovgah8KRKtg,QhbR5TEAMco
3,UCkvK_5omS-42Ovgah8KRKtg,3qYykmHx5gc
4,UCkvK_5omS-42Ovgah8KRKtg,SxixuClI2vA
...,...,...
2588,UCq7EY7H2XF6TV7Z5mLv-aNg,bm-3Qrr7EUA
2589,UCq7EY7H2XF6TV7Z5mLv-aNg,hkxRjDS_Cjw
2590,UCq7EY7H2XF6TV7Z5mLv-aNg,8kElrRAa1ws
2591,UCq7EY7H2XF6TV7Z5mLv-aNg,7x6tX5T7LOs


In [65]:
df_youtuber_videos.to_csv('data/get_youtuber_videos_output/youtuber_videos.csv', index = False)

In [70]:
quotas

534